# Question 6

In [1]:
# To DO 
# implement the activation and its derivative

import numpy as np

def activation(x):
    return 1/(1+np.exp(-x))




def activation_derivative(x):
    return activation(x)*(1-activation(x)) 

In [2]:
# To DO 
# implement the loss its derivative
def compute_loss(predicted_op, true_y):
    loss = 0
    return loss

def loss_derivative(x):
    return 

In [3]:
# TO DO 
# organize the training data into batches
def make_batches(x_train, y_train, batch_size):

    return x_batches, y_batches

In [4]:
# TO DO 
def forward_pass(x_batch, y_batch):

  return predicted_op


In [5]:
# TO DO 
def backward_pass(loss):

  return grads

In [7]:
#TO DO
def update_weights(weights, grads, lr):

  return

In [8]:
# To Do
# define you stopping criteria for training
def stopping_criteria():
  stop = False
  return stop

In [9]:
# TO DO
# Define the hyperparameters. Add any additional hyperparameters you might need
lr = 0  # learning rate
batch_size = 0
num_epochs = 0

In [10]:
def train(x_train, y_train, lr, batch_size, num_epochs):

  while not stopping_criteria():
      # TO DO 
      # get a batch from the data
      x_batch, y_batch = [], []

      # forward the batch through the network layers
      predicted_op = forward_pass(x_batch)

      # compute the loss
      loss = compute_loss(predicted_op, y_batch)

      # perform backward pass
      grads = backward_pass(loss)

      # update the weights of the network
      update_weights(network, grads, lr)
